In [1]:
import requests
from bs4 import BeautifulSoup

# config
from config import genius_access_token

In [2]:
access_token = genius_access_token

In [3]:
def search_song(song_name):
    # Replace 'YOUR_ACCESS_TOKEN' with your Genius API access token
    headers = {'Authorization': 'Bearer ' + access_token}
    base_url = 'https://api.genius.com'
    search_url = base_url + '/search?per_page=10&page=1&q=' + song_name

    try:
        response = requests.get(search_url, headers=headers)
        response.raise_for_status()
        data = response.json()
        
        # Check if any hits were found
        if data['meta']['status'] == 200 and data['response']['hits']:
            # Extract the first hit
            hit = data['response']['hits'][0]['result']
            # Print the page URL
            return hit['url']
        else:
            return None
    except requests.exceptions.RequestException as e:
        print("Error:", e)

In [4]:
def scrape_song_lyrics(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")

    artist_name = (
        soup.select('div[class*="HeaderArtistAndTracklistdesktop__Container-sc-"]')[0]
        .select('a[class*="HeaderArtistAndTracklistdesktop__Artist-sc-"]')[0]
        .get_text()
    )
    song_name = soup.select('h1[class*="SongHeaderdesktop__Title-sc-"]')[0].get_text()

    # print(f"Scraping {song_name} by {artist_name}...")

    lyrics = soup.select('div[class*="Lyrics__Container-sc"]')
    lyrics = "\n".join([lyric.get_text(separator="\n") for lyric in lyrics])

    # print(lyrics)

    return {"song_name": song_name, "artist_name": artist_name, "lyrics": lyrics}

In [5]:
def get_lyrics(song_name):
    url = search_song(song_name)
    if url:
        lyrics = scrape_song_lyrics(url)
        reconstructed = f"{lyrics['artist_name']} - {lyrics['song_name']}"
        print(reconstructed.lower() == song_name.lower())
        return lyrics
    else:
        return None

In [6]:
# get_lyrics("Hikoyuki Sawano - Ymniam - Mkorch")
get_lyrics("5th dimension - one less bell to answer")

False


{'song_name': 'One Less Bell to Answer',
 'artist_name': 'The 5th Dimension',
 'lyrics': "One less bell to answer\nOne less egg to fry\nOne less man to pick up after\nI should be happy\nBut all I do is cry\n(Cry, cry, no more laughter)\nOh, I should be happy\n(Oh, why did he go)\nI only know that since he left\nMy life's so empty\nThough I try to forget\nIt just can't be done\nEach time the doorbell rings\nI still run (I still run)\nI don't know how in the world\nTo stop thinking of him\n'Cause I still love him so\nI end each day the way I start out\nCrying my heart out\nOh, one less man to pick up after\nNo more laughter\nNo more love\nSince he went, oh he went away (He went away)\n(One less bell to answer)\nWhy did he leave me\n(Oh why, why, why did he leave)\n(One less bell to answer)\nNow I've got one less egg to fry\nOne less egg to fry\n(Oh why, why, why did he leave)\nAnd all I do is cry\n(One less bell to answer)\nBecause my man told me goodbye\n(Oh why, why, why did he leave)\